In [8]:
import requests
import bs4
import time
import json
import os.path

In [2]:
root = 'http://www.kijiji.ca/b-cars-trucks/british-columbia/dodge-ram-van/mini+van__other+body+type/k0c174l9007a138?price=10__12000'

In [9]:
if not os.path.exists('kijjivans.json'):
    with open('kijjivans.json', 'w') as fp:
        pass

In [3]:
response = requests.get(root)
soup = bs4.BeautifulSoup(response.text)
links = [tag.find('a')['href'] for tag in 
         soup.find('div', {'class':'container-results'}).find_all('div', {'class':'clearfix'})]
links = ['http://www.kijiji.ca%s' % link for link in links]
print (links)

['http://www.kijiji.ca/v-cars-trucks/kamloops/2002-dodge-power-ram-3500-minivan-van-2-vans-for-price-of-1/1201536808?src=topAdSearch', 'http://www.kijiji.ca/v-cars-trucks/vernon/2001-dodge-ram-van-other/1192131080?src=topAdSearch', 'http://www.kijiji.ca/v-cars-trucks/kamloops/2002-dodge-power-ram-3500-minivan-van-2-vans-for-price-of-1/1201536808', 'http://www.kijiji.ca/v-cars-trucks/vernon/2001-dodge-ram-van-other/1192131080', 'http://www.kijiji.ca/v-cars-trucks/williams-lake/2001-dodge-van-b3500/1200018898', 'http://www.kijiji.ca/v-cars-trucks/vernon/1996-dodge-work-van/1199861427', 'http://www.kijiji.ca/v-cars-trucks/vernon/1990-dodge-b250-ram-van-propane-5-2-l-extended-van-automatic/591605402', 'http://www.kijiji.ca/v-cars-trucks/nelson/1991-dodge-ram-van-150/1196405964', 'http://www.kijiji.ca/v-cars-trucks/whistler/dodge-power-ram-1500-minivan-van-camperized/1194429281', 'http://www.kijiji.ca/v-cars-trucks/nelson/2000-dodge-ram-van-3500-minivan-van/1192911735', 'http://www.kijiji.c

/usr/local/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 1 of the file <string>. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [4]:
for link in links:
    posting = requests.get(link)
    break

In [5]:
print(link)

http://www.kijiji.ca/v-cars-trucks/kamloops/2002-dodge-power-ram-3500-minivan-van-2-vans-for-price-of-1/1201536808?src=topAdSearch


In [6]:
def extract_posting(soup):
    posting = {}
    posting['title'] = soup.find('span', {'itemprop':'name'}).text
    table = soup.find('table', {'class':'ad-attributes'})
    for row in table.find_all('tr'):
        key = row.find('th')
        value = row.find('td')
        if not key or not value: continue
        value = value.text.replace('View map', '')
        posting[key.text.strip()] = ' '.join(value.split())
    images_container = soup.find('ul', {'id' : 'ShownImage'})
    images = [image['src'] for image in images_container.find_all('img')]
    posting['images'] = images
    posting['body'] = ' '.join(soup.find('span', {'itemprop':'description'}).text.split())
    return posting
print (extract_posting(bs4.BeautifulSoup(posting.text)))

{'No. of Seats': '2', 'Make': 'Dodge', 'Model': 'Power Ram 3500', 'Drivetrain': 'Rear-wheel drive (RWD)', 'No. of Doors': '4', 'Date Listed': '26-Sep-16', 'Kilometers': '158,666', 'Colour': 'White', 'Address': '850 Pembroke Ave, Kamloops, BC V2B 1Z3, Canada', 'body': '2 Work Vans for the price of one. $3500 OBO for both!!!!!! Call 250-879-2777 LOW KM ON BOTH! BOTH VANS ARE THE SAME YEAR AND STYLE! ALL THE PARTS MATCH UP!!! First reasonable offer takes them away, need the parking spots NOW! One van runs well, no issues and One of the Vans is an ICBC write off so it cannot be insured, everything runs except the transmission is messed up. Its more of a parts van. VEHICLE OPTIONS: Air Conditioning Cruise control Power steering Tilt wheel CD player Airbag: driver Airbag: passenger', 'Transmission': 'Automatic', 'images': ['http://i.ebayimg.com/00/s/NjAwWDgwMA==/z/LyMAAOSwzaJX4p-b/$_35.JPG', 'http://i.ebayimg.com/00/s/NjAwWDgwMA==/z/~AwAAOSwTA9X4p-T/$_35.JPG', 'http://i.ebayimg.com/00/s/NjAw

/usr/local/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 1 of the file <string>. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [12]:
with open('kijjivans.json') as fp:
    vans = [json.loads(line) for line in fp]
crawled_vans = set([van['url'] for van in vans])

def crawl_searched_vans(links, car, category):
    with open('kijjivans.json', 'a') as fp:
        for url in links:
            if url in crawled_vans: continue
            van = requests.get(url)
            try:
                posting = extract_posting(bs4.BeautifulSoup(van.text))
            except:
                posting = {}
            posting['url'] = url
            posting['query'] = car
            posting['type'] = category
            posting['date'] = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
            posting['deadlink'] = False
            posting['crawler'] = 'Kijji'
            crawled_vans.add(url)
            fp.write(json.dumps(posting) + '\n')
            time.sleep(5)
            
def crawl_all_vans():        
    category_link = {'van': 'http://www.kijiji.ca/b-cars-trucks/british-columbia/%s/mini+van__other+body+type/k0c174l9007a138?price=10__12000',
             'rv' : 'http://www.kijiji.ca/b-rv-camper-trailer/british-columbia/%s/k0c172l9007?price=10__12000'}
    cars = ['dodge ram van', 'ford tuscany van', 'food truck', 'chevy express', 'gmc savana', 'ford econoline', 
            'ford e350', 'ford transit', 'cargo van' ]
    cars = [car.replace(' ', '-') for car in cars]

    for category in category_link:
        for car in cars:
            search_page = category_link[category] % car
            print (search_page, end=": ")
            response = requests.get(search_page)

            soup = bs4.BeautifulSoup(response.text)
            try:
                links = [tag.find('a')['href'] for tag in 
                         soup.find('div', {'class':'container-results'}).find_all('div', {'class':'clearfix'})]
                links = ['http://www.kijiji.ca%s' % link for link in links]
                print (len(links))
                time.sleep(3)
                crawl_searched_vans(links, car, category)
            except: 
                print (0)
                pass

In [13]:
crawl_all_vans()

http://www.kijiji.ca/b-cars-trucks/british-columbia/dodge-ram-van/mini+van__other+body+type/k0c174l9007a138?price=10__12000: 

/usr/local/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 1 of the file <string>. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


14
http://www.kijiji.ca/b-cars-trucks/british-columbia/ford-tuscany-van/mini+van__other+body+type/k0c174l9007a138?price=10__12000: 0
http://www.kijiji.ca/b-cars-trucks/british-columbia/food-truck/mini+van__other+body+type/k0c174l9007a138?price=10__12000: 0
http://www.kijiji.ca/b-cars-trucks/british-columbia/chevy-express/mini+van__other+body+type/k0c174l9007a138?price=10__12000: 1
http://www.kijiji.ca/b-cars-trucks/british-columbia/gmc-savana/mini+van__other+body+type/k0c174l9007a138?price=10__12000: 4
http://www.kijiji.ca/b-cars-trucks/british-columbia/ford-econoline/mini+van__other+body+type/k0c174l9007a138?price=10__12000: 16
http://www.kijiji.ca/b-cars-trucks/british-columbia/ford-e350/mini+van__other+body+type/k0c174l9007a138?price=10__12000: 21
http://www.kijiji.ca/b-cars-trucks/british-columbia/ford-transit/mini+van__other+body+type/k0c174l9007a138?price=10__12000: 5
http://www.kijiji.ca/b-cars-trucks/british-columbia/cargo-van/mini+van__other+body+type/k0c174l9007a138?price=10_